In [100]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import os

# Calculate the how much men and women would need to save for a comfortable retirement in each state, then calculate the difference


## Take the average retirement age of 64.

In [2]:
avg_retirement = 64

## Take the most recent life expectancies for each gender in each state

https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_life_expectancy

In [3]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_life_expectancy')

In [5]:
life_expectancies = tables[0]

In [8]:
new_columns = ['state', 'overall_2019', 'males_2019', 'females_2019', 'F Δ M 2019', 
               '2019→2020', 'overall_2020', 'males_2020', 'females_2020', 'F Δ M 2020']

In [9]:
life_expectancies.columns = new_columns

In [11]:
life_expectancies.drop(columns=['overall_2019', 'males_2019', 'females_2019', 
                                'F Δ M 2019', '2019→2020',], inplace=True)

In [12]:
life_expectancies

,state,overall_2020,males_2020,females_2020,F Δ M 2020
0,US on average,77.0,76.2,79.9,3.7
1,California,79.0,76.2,82.0,5.8
2,Hawaii,80.7,77.6,83.8,6.2
3,New York,77.7,74.8,80.7,5.9
4,Minnesota,79.1,76.8,81.4,4.6
5,Massachusetts,79.0,76.4,81.5,5.1
6,Connecticut,78.4,75.6,81.3,5.7
7,New Jersey,77.5,74.6,80.5,5.9
8,Washington (state),79.2,76.9,81.6,4.7
9,Colorado,78.3,75.8,80.9,5.1


## Take the cost of living in each state from MIT Source

https://livingwage.mit.edu/states/48

In [48]:
living_cost = pd.DataFrame()

In [14]:
urls = []

In [17]:
for i in range(1,52):
    url = f'https://livingwage.mit.edu/states/{i}'
    urls.append(url)

In [18]:
urls

['https://livingwage.mit.edu/states/1',
 'https://livingwage.mit.edu/states/2',
 'https://livingwage.mit.edu/states/3',
 'https://livingwage.mit.edu/states/4',
 'https://livingwage.mit.edu/states/5',
 'https://livingwage.mit.edu/states/6',
 'https://livingwage.mit.edu/states/7',
 'https://livingwage.mit.edu/states/8',
 'https://livingwage.mit.edu/states/9',
 'https://livingwage.mit.edu/states/10',
 'https://livingwage.mit.edu/states/11',
 'https://livingwage.mit.edu/states/12',
 'https://livingwage.mit.edu/states/13',
 'https://livingwage.mit.edu/states/14',
 'https://livingwage.mit.edu/states/15',
 'https://livingwage.mit.edu/states/16',
 'https://livingwage.mit.edu/states/17',
 'https://livingwage.mit.edu/states/18',
 'https://livingwage.mit.edu/states/19',
 'https://livingwage.mit.edu/states/20',
 'https://livingwage.mit.edu/states/21',
 'https://livingwage.mit.edu/states/22',
 'https://livingwage.mit.edu/states/23',
 'https://livingwage.mit.edu/states/24',
 'https://livingwage.mit.

In [52]:
for url in urls[2:]:
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text)

        state = soup.find_all('h1')[-1].text.split(' ')[-1]
        
        tables = soup.find_all('table')
        second_table = tables[1]
        rows = second_table.find_all('tr')
        
        data = []
        for row in rows:
            cols = row.find_all('td')
            cols = [col.text.strip() for col in cols]
            data.append(cols)
            
        data = pd.DataFrame(data)
        data = data[[0, 1]]
        data['state'] = state
        
        living_cost = pd.concat([living_cost, data])
        print('Done', state)
        time.sleep(5)
    except Exception as e:
            print(f'Exception: {e}')

Exception: 404 Client Error: Not Found for url: https://livingwage.mit.edu/states/3
Done Arizona
Done Arkansas
Done California
Exception: 404 Client Error: Not Found for url: https://livingwage.mit.edu/states/7
Done Colorado
Done Connecticut
Done Delaware
Done Columbia
Done Florida
Done Georgia
Exception: 404 Client Error: Not Found for url: https://livingwage.mit.edu/states/14
Done Hawaii
Done Idaho
Done Illinois
Done Indiana
Done Iowa
Done Kansas
Done Kentucky
Done Louisiana
Done Maine
Done Maryland
Done Massachusetts
Done Michigan
Done Minnesota
Done Mississippi
Done Missouri
Done Montana
Done Nebraska
Done Nevada
Done Hampshire
Done Jersey
Done Mexico
Done York
Done Carolina
Done Dakota
Done Ohio
Done Oklahoma
Done Oregon
Done Pennsylvania
Exception: 404 Client Error: Not Found for url: https://livingwage.mit.edu/states/43
Done Island
Done Carolina
Done Dakota
Done Tennessee
Done Texas
Done Utah
Done Vermont
Done Virginia


In [54]:
living_cost.columns = ['expense', '$', 'state']

In [56]:
living_cost = living_cost[living_cost['expense'] == 'Required annual income before taxes']

In [58]:
living_cost = living_cost[['$', 'state']]

In [59]:
living_cost.columns = ['Required annual income before taxes', 'state']

In [61]:
living_cost = living_cost[['state', 'Required annual income before taxes']]

In [66]:
new_rows_df = pd.DataFrame({
    'state': ['Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
    'Required annual income before taxes': ['$53,242', '$39,386', '$42,062', '$43,823']
})

In [67]:
living_cost = pd.concat([living_cost, new_rows_df], ignore_index=True)

In [69]:
#living_cost.to_csv('living_cost.csv', index=False)

In [133]:
living_cost = pd.read_csv('living_cost_new.csv')

In [136]:
living_cost

,state,Required annual income before taxes
0,Alabama,41911
1,Alaska,48375
2,Arizona,48677
3,Arkansas,39724
4,California,56825
5,Colorado,51644
6,Connecticut,50194
7,Delaware,47064
8,Columbia,49714
9,Florida,46645


## Multiply this annual figure for each state by 20% = more comfortable retirement


In [135]:
living_cost['Required annual income before taxes'] = living_cost['Required annual income before taxes'].apply(lambda x: int(x.replace('$', '').replace(',', '')))


In [86]:
living_cost['Required annual income before taxes'] = living_cost['Required annual income before taxes'].apply(lambda x: x*1.2)

In [87]:
living_cost

,state,Required annual income before taxes
0,Alabama,50293.2
1,Alaska,58050.0
2,Arizona,58412.4
3,Arkansas,47668.8
4,California,68190.0
5,Colorado,61972.8
6,Connecticut,60232.8
7,Delaware,56476.8
8,Columbia,59656.8
9,Florida,55974.0


## Calculate the difference in years between the retirement age and life expectancy for each gender in each state.

In [76]:
life_expectancies = life_expectancies[['state', 'males_2020', 'females_2020']]

In [78]:
life_expectancies['Years_After_Retirement_Male'] = life_expectancies['males_2020'].apply(lambda x: x - avg_retirement)

/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
life_expectancies['Years_After_Retirement_Female'] = life_expectancies['females_2020'].apply(lambda x: x - avg_retirement)

In [81]:
life_expectancies

,state,males_2020,females_2020,Years_After_Retirement_Male,Years_After_Retirement_Female
0,US on average,76.2,79.9,12.2,15.9
1,California,76.2,82.0,12.2,18.0
2,Hawaii,77.6,83.8,13.6,19.8
3,New York,74.8,80.7,10.8,16.7
4,Minnesota,76.8,81.4,12.8,17.4
5,Massachusetts,76.4,81.5,12.4,17.5
6,Connecticut,75.6,81.3,11.6,17.3
7,New Jersey,74.6,80.5,10.6,16.5
8,Washington (state),76.9,81.6,12.9,17.6
9,Colorado,75.8,80.9,11.8,16.9


## Calculate the savings needed to retire for both genders in each state

In [92]:
life_expectancies = life_expectancies.merge(living_cost, on=['state'], how='left').dropna(subset = ['Required annual income before taxes'])

In [94]:
life_expectancies['Savings_Male'] = life_expectancies['Years_After_Retirement_Male'] * life_expectancies['Required annual income before taxes']

In [96]:
life_expectancies['Savings_Female'] = life_expectancies['Years_After_Retirement_Female'] * life_expectancies['Required annual income before taxes']

In [97]:
life_expectancies

,state,males_2020,females_2020,Years_After_Retirement_Male,Years_After_Retirement_Female,Required annual income before taxes,Savings_Male,Savings_Female
1,California,76.2,82.0,12.2,18.0,68190.0,831918.00,1227420.00
2,Hawaii,77.6,83.8,13.6,19.8,68209.2,927645.12,1350542.16
3,New York,74.8,80.7,10.8,16.7,67053.6,724178.88,1119795.12
4,Minnesota,76.8,81.4,12.8,17.4,53551.2,685455.36,931790.88
5,Massachusetts,76.4,81.5,12.4,17.5,69610.8,863173.92,1218189.00
6,Connecticut,75.6,81.3,11.6,17.3,60232.8,698700.48,1042027.44
7,New Jersey,74.6,80.5,10.6,16.5,61804.8,655130.88,1019779.20
8,Washington (state),76.9,81.6,12.9,17.6,63890.4,824186.16,1124471.04
9,Colorado,75.8,80.9,11.8,16.9,61972.8,731279.04,1047340.32
10,Vermont,76.1,81.4,12.1,17.4,57470.4,695391.84,999984.96


## Calculate how much more women would need to save in each state.

In [98]:
life_expectancies['Extra_Savings_for_Women'] = life_expectancies['Savings_Female'] - life_expectancies['Savings_Male']

In [152]:
life_expectancies.columns = ['state', 'males_2020', 'females_2020', 'Years_After_Retirement_Male',
       'Years_After_Retirement_Female', 'Required annual income before taxes * 20%',
       'Savings_Male', 'Savings_Female', 'Extra_Savings_for_Women']

In [153]:
life_expectancies

,state,males_2020,females_2020,Years_After_Retirement_Male,Years_After_Retirement_Female,Required annual income before taxes * 20%,Savings_Male,Savings_Female,Extra_Savings_for_Women
1,California,76.2,82.0,12.2,18.0,68190.0,831918.00,1227420.00,395502.00
2,Hawaii,77.6,83.8,13.6,19.8,68209.2,927645.12,1350542.16,422897.04
3,New York,74.8,80.7,10.8,16.7,67053.6,724178.88,1119795.12,395616.24
4,Minnesota,76.8,81.4,12.8,17.4,53551.2,685455.36,931790.88,246335.52
5,Massachusetts,76.4,81.5,12.4,17.5,69610.8,863173.92,1218189.00,355015.08
6,Connecticut,75.6,81.3,11.6,17.3,60232.8,698700.48,1042027.44,343326.96
7,New Jersey,74.6,80.5,10.6,16.5,61804.8,655130.88,1019779.20,364648.32
8,Washington (state),76.9,81.6,12.9,17.6,63890.4,824186.16,1124471.04,300284.88
9,Colorado,75.8,80.9,11.8,16.9,61972.8,731279.04,1047340.32,316061.28
10,Vermont,76.1,81.4,12.1,17.4,57470.4,695391.84,999984.96,304593.12


# Calculate the years men and women would need to work to save for a comfortable retirement in each state, then calculate the difference

## Pull the median earnings for men and women in each state

https://data.census.gov/table/ACSST1Y2022.S2412?q=s2412&moe=false

In [101]:
folder_path = '/Users/anna.zemit/Desktop/Programming/2024/24. How Much More Do Women Need to Save For A Comfortable Retirement?/median earnings'

In [119]:
# List to store individual DataFrames
dfs = []

In [120]:
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Extract the state name from the header row (assumed to be in the first row or specific header)
        state_name = df.columns[1].split('!!')[0].strip()  # Modify the splitting method if the state name format is different
        
        # Standardize column names by removing state name and keeping only the column description
        df.columns = [col.split('!!')[-1].strip() for col in df.columns]
        
        # Add the state name as a new column to the DataFrame
        df['state'] = state_name
        
        # Append the DataFrame to the list
        dfs.append(df)


In [121]:
median_earnings = pd.concat(dfs, ignore_index=True)

In [122]:
len(median_earnings.state.unique())

50

In [125]:
median_earnings.columns = ['Label', 'Median earnings', 'Median earnings for male',
                          'Median earnings for female', 'Women\'s earnings as a percentage of men\'s earning',
                          'state']

In [128]:
median_earnings = median_earnings[median_earnings.Label == 'Full-time, year-round civilian employed population 16 years and over with earnings']

In [130]:
median_earnings = median_earnings[['Median earnings for male', 'Median earnings for female', 'state']]

In [138]:
median_earnings.loc[median_earnings['state'] == 'Washington', 'state'] =  'Washington (state)'

/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [139]:
median_earnings

,Median earnings for male,Median earnings for female,state
0,"63,639","52,329",Hawaii
36,"55,041","42,616",West Virginia
72,"60,111","46,345",Indiana
108,"67,414","56,020",Minnesota
144,"63,636","52,154",Pennsylvania
180,"75,042","63,833",Maryland
216,"53,768","45,993",New Mexico
252,"62,192","48,084",North Dakota
288,"61,551","54,972",Vermont
324,"66,112","53,350",Illinois


## Subtract the cost of living for each state from each gender’s median earnings to calculate the estimated yearly savings for each gender in each state.

In [141]:
median_earnings = median_earnings.merge(living_cost, on=['state'], how='left')

In [144]:
median_earnings['Median earnings for male'] = median_earnings['Median earnings for male'].apply(lambda x: int(x.replace(',', '')))

In [145]:
median_earnings['Median earnings for female'] = median_earnings['Median earnings for female'].apply(lambda x: int(x.replace(',', '')))

In [146]:
median_earnings['Estimated_yearly_savings_Male'] = median_earnings['Median earnings for male'] - median_earnings['Required annual income before taxes']

In [148]:
median_earnings['Estimated_yearly_savings_Female'] = median_earnings['Median earnings for female'] - median_earnings['Required annual income before taxes']

In [149]:
median_earnings

,Median earnings for male,Median earnings for female,state,Required annual income before taxes,Estimated_yearly_savings_Male,Estimated_yearly_savings_Female
0,63639,52329,Hawaii,56841,6798,-4512
1,55041,42616,West Virginia,39386,15655,3230
2,60111,46345,Indiana,42510,17601,3835
3,67414,56020,Minnesota,44626,22788,11394
4,63636,52154,Pennsylvania,45661,17975,6493
5,75042,63833,Maryland,51460,23582,12373
6,53768,45993,New Mexico,41807,11961,4186
7,62192,48084,North Dakota,40262,21930,7822
8,61551,54972,Vermont,47892,13659,7080
9,66112,53350,Illinois,47559,18553,5791


## Calculate how many years each gender would need to work to save for the comfortable retirement

In [155]:
df = life_expectancies.merge(median_earnings, on=['state'], how='left')

In [157]:
df['Years_Worked_for_Retirement_Male'] = df['Savings_Male']/df['Estimated_yearly_savings_Male']

In [159]:
df['Years_Worked_for_Retirement_Female'] = df['Savings_Female']/df['Estimated_yearly_savings_Female']

In [162]:
df.columns = ['state', 'males_life_expectation', 'females_life_expectation', 
              'Years_After_Retirement_Male','Years_After_Retirement_Female',
       'Required annual income before taxes * 20%', 'Savings_Male',
       'Savings_Female', 'Extra_Savings_for_Women', 'Median earnings for male',
       'Median earnings for female', 'Required annual income before taxes',
       'Estimated_yearly_savings_Male', 'Estimated_yearly_savings_Female',
       'Years_Worked_for_Retirement_Male',
       'Years_Worked_for_Retirement_Female']

In [163]:
df

,state,males_life_expectation,females_life_expectation,Years_After_Retirement_Male,Years_After_Retirement_Female,Required annual income before taxes * 20%,Savings_Male,Savings_Female,Extra_Savings_for_Women,Median earnings for male,Median earnings for female,Required annual income before taxes,Estimated_yearly_savings_Male,Estimated_yearly_savings_Female,Years_Worked_for_Retirement_Male,Years_Worked_for_Retirement_Female
0,California,76.2,82.0,12.2,18.0,68190.0,831918.00,1227420.00,395502.00,67700,59731,56825,10875,2906,76.498207,422.374398
1,Hawaii,77.6,83.8,13.6,19.8,68209.2,927645.12,1350542.16,422897.04,63639,52329,56841,6798,-4512,136.458535,-299.322287
2,New York,74.8,80.7,10.8,16.7,67053.6,724178.88,1119795.12,395616.24,69668,60990,55878,13790,5112,52.514785,219.052254
3,Minnesota,76.8,81.4,12.8,17.4,53551.2,685455.36,931790.88,246335.52,67414,56020,44626,22788,11394,30.079663,81.779084
4,Massachusetts,76.4,81.5,12.4,17.5,69610.8,863173.92,1218189.00,355015.08,79976,67222,58009,21967,9213,39.294119,132.225008
5,Connecticut,75.6,81.3,11.6,17.3,60232.8,698700.48,1042027.44,343326.96,75429,63977,50194,25235,13783,27.687754,75.602368
6,New Jersey,74.6,80.5,10.6,16.5,61804.8,655130.88,1019779.20,364648.32,76375,63414,51504,24871,11910,26.341156,85.623778
7,Washington (state),76.9,81.6,12.9,17.6,63890.4,824186.16,1124471.04,300284.88,76926,62111,53242,23684,8869,34.799281,126.786677
8,Colorado,75.8,80.9,11.8,16.9,61972.8,731279.04,1047340.32,316061.28,71428,57067,51644,19784,5423,36.963154,193.129323
9,Vermont,76.1,81.4,12.1,17.4,57470.4,695391.84,999984.96,304593.12,61551,54972,47892,13659,7080,50.910890,141.240814


# Finalise the data into tabs

## States Map - Savings

Rank all states based on the extra money that women need to save.

In [165]:
States_Map_Savings = df.sort_values(by=['Extra_Savings_for_Women'], ascending=False)

In [166]:
States_Map_Savings

,state,males_life_expectation,females_life_expectation,Years_After_Retirement_Male,Years_After_Retirement_Female,Required annual income before taxes * 20%,Savings_Male,Savings_Female,Extra_Savings_for_Women,Median earnings for male,Median earnings for female,Required annual income before taxes,Estimated_yearly_savings_Male,Estimated_yearly_savings_Female,Years_Worked_for_Retirement_Male,Years_Worked_for_Retirement_Female
1,Hawaii,77.6,83.8,13.6,19.8,68209.2,927645.12,1350542.16,422897.04,63639,52329,56841,6798,-4512,136.458535,-299.322287
2,New York,74.8,80.7,10.8,16.7,67053.6,724178.88,1119795.12,395616.24,69668,60990,55878,13790,5112,52.514785,219.052254
0,California,76.2,82.0,12.2,18.0,68190.0,831918.00,1227420.00,395502.00,67700,59731,56825,10875,2906,76.498207,422.374398
6,New Jersey,74.6,80.5,10.6,16.5,61804.8,655130.88,1019779.20,364648.32,76375,63414,51504,24871,11910,26.341156,85.623778
24,Maryland,73.8,79.7,9.8,15.7,61752.0,605169.60,969506.40,364336.80,75042,63833,51460,23582,12373,25.662353,78.356615
22,Arizona,73.4,79.5,9.4,15.5,58412.4,549076.56,905392.20,356315.64,58144,50245,48677,9467,1568,57.999003,577.418495
4,Massachusetts,76.4,81.5,12.4,17.5,69610.8,863173.92,1218189.00,355015.08,79976,67222,58009,21967,9213,39.294119,132.225008
5,Connecticut,75.6,81.3,11.6,17.3,60232.8,698700.48,1042027.44,343326.96,75429,63977,50194,25235,13783,27.687754,75.602368
19,Illinois,73.8,79.8,9.8,15.8,57070.8,559293.84,901718.64,342424.80,66112,53350,47559,18553,5791,30.145736,155.710351
41,South Carolina,71.7,78.0,7.7,14.0,52982.4,407964.48,741753.60,333789.12,56016,43865,44152,11864,-287,34.386757,-2584.507317


## Table - Savings

All data for all states, with columns for men savings, women savings and the difference.


In [167]:
table_savings = df.copy()

In [168]:
table_savings

,state,males_life_expectation,females_life_expectation,Years_After_Retirement_Male,Years_After_Retirement_Female,Required annual income before taxes * 20%,Savings_Male,Savings_Female,Extra_Savings_for_Women,Median earnings for male,Median earnings for female,Required annual income before taxes,Estimated_yearly_savings_Male,Estimated_yearly_savings_Female,Years_Worked_for_Retirement_Male,Years_Worked_for_Retirement_Female
0,California,76.2,82.0,12.2,18.0,68190.0,831918.00,1227420.00,395502.00,67700,59731,56825,10875,2906,76.498207,422.374398
1,Hawaii,77.6,83.8,13.6,19.8,68209.2,927645.12,1350542.16,422897.04,63639,52329,56841,6798,-4512,136.458535,-299.322287
2,New York,74.8,80.7,10.8,16.7,67053.6,724178.88,1119795.12,395616.24,69668,60990,55878,13790,5112,52.514785,219.052254
3,Minnesota,76.8,81.4,12.8,17.4,53551.2,685455.36,931790.88,246335.52,67414,56020,44626,22788,11394,30.079663,81.779084
4,Massachusetts,76.4,81.5,12.4,17.5,69610.8,863173.92,1218189.00,355015.08,79976,67222,58009,21967,9213,39.294119,132.225008
5,Connecticut,75.6,81.3,11.6,17.3,60232.8,698700.48,1042027.44,343326.96,75429,63977,50194,25235,13783,27.687754,75.602368
6,New Jersey,74.6,80.5,10.6,16.5,61804.8,655130.88,1019779.20,364648.32,76375,63414,51504,24871,11910,26.341156,85.623778
7,Washington (state),76.9,81.6,12.9,17.6,63890.4,824186.16,1124471.04,300284.88,76926,62111,53242,23684,8869,34.799281,126.786677
8,Colorado,75.8,80.9,11.8,16.9,61972.8,731279.04,1047340.32,316061.28,71428,57067,51644,19784,5423,36.963154,193.129323
9,Vermont,76.1,81.4,12.1,17.4,57470.4,695391.84,999984.96,304593.12,61551,54972,47892,13659,7080,50.910890,141.240814


## States Map - Years

States Map - Years: rank all states based on the extra years women need to work.

In [169]:
States_Map_Years = df.sort_values(by=['Years_Worked_for_Retirement_Female'], ascending=False)

In [170]:
States_Map_Years

,state,males_life_expectation,females_life_expectation,Years_After_Retirement_Male,Years_After_Retirement_Female,Required annual income before taxes * 20%,Savings_Male,Savings_Female,Extra_Savings_for_Women,Median earnings for male,Median earnings for female,Required annual income before taxes,Estimated_yearly_savings_Male,Estimated_yearly_savings_Female,Years_Worked_for_Retirement_Male,Years_Worked_for_Retirement_Female
31,Nevada,73.5,79.2,9.5,15.2,56072.4,532687.80,852300.48,319612.68,54036,46802,46727,7309,75,72.881078,11364.006400
47,Alabama,70.1,76.4,6.1,12.4,50293.2,306788.52,623635.68,316847.16,56038,42006,41911,14127,95,21.716466,6564.586105
36,Georgia,72.8,78.3,8.8,14.3,58137.6,511610.88,831367.68,319756.80,59870,48581,48448,11422,133,44.791707,6250.884812
10,Utah,76.7,80.6,12.7,16.6,56220.0,713994.00,933252.00,219258.00,65512,47907,46850,18662,1057,38.259243,882.925260
44,Louisiana,69.9,76.4,5.9,12.4,49479.6,291929.64,613547.04,321617.40,56245,42044,41233,15012,811,19.446419,756.531492
22,Arizona,73.4,79.5,9.4,15.5,58412.4,549076.56,905392.20,356315.64,58144,50245,48677,9467,1568,57.999003,577.418495
0,California,76.2,82.0,12.2,18.0,68190.0,831918.00,1227420.00,395502.00,67700,59731,56825,10875,2906,76.498207,422.374398
43,Arkansas,71.1,76.6,7.1,12.6,47668.8,338448.48,600626.88,262178.40,51383,41205,39724,11659,1481,29.028946,405.554949
35,North Carolina,73.3,79.0,9.3,15.0,53817.6,500503.68,807264.00,306760.32,56210,47188,44848,11362,2340,44.050667,344.984615
45,Tennessee,70.7,77.0,6.7,13.0,51835.2,347295.84,673857.60,326561.76,55437,45150,43196,12241,1954,28.371525,344.860594


## Table - Years

Table - Years: all data for all states, with columns for men years, women years and the difference.

In [171]:
Table_Years = df.copy()

In [172]:
Table_Years['Extra_Years_for_Women'] = Table_Years['Years_Worked_for_Retirement_Female'] - Table_Years['Years_Worked_for_Retirement_Male']

In [173]:
Table_Years

,state,males_life_expectation,females_life_expectation,Years_After_Retirement_Male,Years_After_Retirement_Female,Required annual income before taxes * 20%,Savings_Male,Savings_Female,Extra_Savings_for_Women,Median earnings for male,Median earnings for female,Required annual income before taxes,Estimated_yearly_savings_Male,Estimated_yearly_savings_Female,Years_Worked_for_Retirement_Male,Years_Worked_for_Retirement_Female,Extra_Years_for_Women
0,California,76.2,82.0,12.2,18.0,68190.0,831918.00,1227420.00,395502.00,67700,59731,56825,10875,2906,76.498207,422.374398,345.876191
1,Hawaii,77.6,83.8,13.6,19.8,68209.2,927645.12,1350542.16,422897.04,63639,52329,56841,6798,-4512,136.458535,-299.322287,-435.780822
2,New York,74.8,80.7,10.8,16.7,67053.6,724178.88,1119795.12,395616.24,69668,60990,55878,13790,5112,52.514785,219.052254,166.537469
3,Minnesota,76.8,81.4,12.8,17.4,53551.2,685455.36,931790.88,246335.52,67414,56020,44626,22788,11394,30.079663,81.779084,51.699421
4,Massachusetts,76.4,81.5,12.4,17.5,69610.8,863173.92,1218189.00,355015.08,79976,67222,58009,21967,9213,39.294119,132.225008,92.930889
5,Connecticut,75.6,81.3,11.6,17.3,60232.8,698700.48,1042027.44,343326.96,75429,63977,50194,25235,13783,27.687754,75.602368,47.914614
6,New Jersey,74.6,80.5,10.6,16.5,61804.8,655130.88,1019779.20,364648.32,76375,63414,51504,24871,11910,26.341156,85.623778,59.282623
7,Washington (state),76.9,81.6,12.9,17.6,63890.4,824186.16,1124471.04,300284.88,76926,62111,53242,23684,8869,34.799281,126.786677,91.987397
8,Colorado,75.8,80.9,11.8,16.9,61972.8,731279.04,1047340.32,316061.28,71428,57067,51644,19784,5423,36.963154,193.129323,156.166169
9,Vermont,76.1,81.4,12.1,17.4,57470.4,695391.84,999984.96,304593.12,61551,54972,47892,13659,7080,50.910890,141.240814,90.329924


# Create an Excel file

In [174]:
# Create an Excel writer object

writer = pd.ExcelWriter('How Much More Do Women Need to Save For A Comfortable Retirement?.xlsx')

In [175]:
States_Map_Savings.to_excel(writer, sheet_name='States Map Savings', index=False)
table_savings.to_excel(writer, sheet_name='Table Savings', index=False)
States_Map_Years.to_excel(writer, sheet_name='States Map Years', index=False)
Table_Years.to_excel(writer, sheet_name='Table Years', index=False)


In [176]:
# Save the Excel file
writer.save()